<a href="https://colab.research.google.com/github/JRasmusBm/chatbot-epsilon/blob/master/Trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is the file in which we perform training of the NN

# Load Data

## In Colab

In [1]:
#from google.colab import files
#uploaded = files.upload()
#file_name = "amazon_cells_labelled.txt"
#uploaded[file_name].decode("utf-8")

## Locally

In [2]:
data_folder = "../../data"
file_name = f"{data_folder}/amazon_cells_labelled.txt"
json_file = f"{data_folder}/amazon_cells_labelled.json"

# Import code (from TA)

In [3]:
#from pprint import pprint
#from IPython.core.debugger import set_trace
#import time

In [4]:
#import torch
#import torch.nn as nn
#import torch.optim as optim
#import torch.nn.functional as F
#from torch.utils.data import TensorDataset, DataLoader
#import numpy as np
#from matplotlib import pyplot
#import pandas as pd
#from sklearn.model_selection import train_test_split
#from sklearn.feature_extraction.text import TfidfVectorizer
#import nltk
#from nltk.corpus import stopwords
#from nltk import word_tokenize
#from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, classification_report
#nltk.download('punkt')
#nltk.download('stopwords')

In [5]:
#def preprocess_pandas(data, columns):
#    df_ = pd.DataFrame(columns=columns)
#    data['Sentence'] = data['Sentence'].str.lower()
#    data['Sentence'] = data['Sentence'].replace('[a-zA-Z0-9-_.]+@[a-zA-Z0-9-_.]+', '', regex=True)                      # remove emails
#    data['Sentence'] = data['Sentence'].replace('((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.|$)){4}', '', regex=True)    # remove IP address
#    data['Sentence'] = data['Sentence'].str.replace('[^\w\s]','')                                                       # remove special characters
#    data['Sentence'] = data['Sentence'].replace('\d', '', regex=True)                                                   # remove numbers
#    for index, row in data.iterrows():
#        word_tokens = word_tokenize(row['Sentence'])
#        filtered_sent = [w for w in word_tokens if not w in stopwords.words('english')]
#        df_ = df_.append({
#            "index": row['index'],
#            "Class": row['Class'],
#            "Sentence": " ".join(filtered_sent[0:])
#        }, ignore_index=True)
#    return data
#
## If this is the primary file that is executed (ie not an import of another file)
#if __name__ == "__main__":
#    # get data, pre-process and split
#    data = pd.read_csv(file_name, delimiter='\t', header=None)
#    data.columns = ['Sentence', 'Class']
#    data['index'] = data.index                                          # add new column index
#    columns = ['index', 'Class', 'Sentence']
#    data = preprocess_pandas(data, columns)                             # pre-process
#    training_data, validation_data, training_labels, validation_labels = train_test_split( # split the data into training, validation, and test splits
#        data['Sentence'].values.astype('U'),
#        data['Class'].values.astype('int32'),
#        test_size=0.10,
#        random_state=0,
#        shuffle=True
#    )
#
#    # vectorize data using TFIDF and transform for PyTorch for scalability
#    word_vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,2), max_features=50000, max_df=0.5, use_idf=True, norm='l2')
#    training_data = word_vectorizer.fit_transform(training_data)        # transform texts to sparse matrix
#    training_data = training_data.todense()                             # convert to dense matrix for Pytorch
#    vocab_size = len(word_vectorizer.vocabulary_)
#    validation_data = word_vectorizer.transform(validation_data)
#    validation_data = validation_data.todense()
#    train_x_tensor = torch.from_numpy(np.array(training_data)).type(torch.FloatTensor)
#    train_y_tensor = torch.from_numpy(np.array(training_labels)).long()
#    validation_x_tensor = torch.from_numpy(np.array(validation_data)).type(torch.FloatTensor)
#    validation_y_tensor = torch.from_numpy(np.array(validation_labels)).long()

In [6]:
#print(len(train_x_tensor[0]))
#print(vocab_size)
##pprint(list(zip(train_x_tensor, train_y_tensor)))

In [7]:
#data.head()

In [8]:
#words = [ 
#    word 
#    for line in data["Sentence"] 
#    for word in line.split(" ") if word != ""
#]
#word_counts = {}
#for line in data["Sentence"]:
#    for word in line.split(" ") :
#        if word == "":
#            continue
#        if word in word_counts:
#            word_counts[word] += 1
#        else:
#            word_counts[word] = 1
#    
#unique_words = set(words)
#prevalent_words = sorted(word_counts.items(), key=lambda item: -item[1])

# Imports

In [9]:
from torchtext import data
import csv

# Extract Data


First, we create lists of labels and sentences. The indices in the one correspond to those in the other. Due to restrictions in torchtext, write it as json to disk.

In [50]:
with open(file_name) as f:
    contents = f.read()
labels = []
sentences = []
for line in (l for l in contents.split("\n") if l):
    labels.append(int(line[-1]))
    sentences.append(str.strip(line[:-1]).replace(
        ".", "").replace(
        "!", "").replace(
        "?", "").replace(
        ",", "").split(" "))
data_json = [
    dict(label=label, sentence=sentence)
    for label, sentence in zip(labels, sentences)
]
with open(json_file, "w") as f:
    text = "\n".join(json.dumps(line) for line in data_json)
    f.write(text)
    

Now we convert the lists of labels into a torchtext dataset.

In [66]:
import random
from torch.utils.data.dataset import random_split
SEED = 1234
pairs = list(zip(sentences, labels))
fields=dict(
    sentence=("sentence", data.Field(sequential=True)), 
    label=("label", data.Field(sequential=False)),
)
from IPython.core.debugger import set_trace
dataset = data.TabularDataset(
    path=json_file,
    format="json",
    fields=fields,
) 
#help(dataset)
training_data, test_data, validation_data = dataset.split(
    split_ratio=[0.7, 0.2, 0.1],
    random_state=random.seed(SEED)
)

print(f"Length (Training Data): {len(training_data)}")
print(f"Length (Test Data): {len(test_data)}")
print(f"Length (Validation Data): {len(validation_data)}")

Length (Training Data): 700
Length (Test Data): 100
Length (Validation Data): 200


# Build RNN

In [12]:
class RNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        output, hidden = self.rnn(text, 1)
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

NameError: name 'nn' is not defined

In [ ]:
model = RNN(input_dim=vocab_size, hidden_dim=256, output_dim=1)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"There are {count_parameters(model):,} trainable parameters in the model")

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train(model, sentences, labels, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for sentence, label in zip(sentences, labels):
        optimizer.zero_grad()
        predictions = model(sentence).squeeze(1)
        loss = criterion(predictions, label)
        acc = binary_accuracy(predictions, label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval() 
    with torch.no_grad(): 
        for sentence, label in iterator: 
            predictions = model(sentence).squeeze(1) 
            loss = criterion(predictions, label) 
            acc = binary_accuracy(predictions, label) 
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss, train_acc = train(model, train_x_tensor, train_y_tensor, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_x_tensor, valid_y_tensor, criterion)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')